In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

import joblib

import os

In [2]:
# Tìm đường dẫn thư mục con và thư mục chính
current_dir = os.path.abspath(os.getcwd())
parrent_dir = os.path.dirname(current_dir)
base_dir = os.path.dirname(parrent_dir)

In [3]:
meta_data = pd.read_csv(base_dir + "\\Dữ liệu đã được xử lý\\meta_data.csv")
meta_data.head()

,category,title,brand,asin
0,Guiros,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,WADSUWAN SHOP,0000989983
1,Sound Effects,"Wooden Percussion 3 Piece Set Frog, Cricket an...",WADSUWAN SHOP,0000098906
2,General Accessories,Hal Leonard Vivaldi Four Seasons for Piano (Or...,Hal Leonard,0041291905
3,General Accessories,Wiley Publishers Crocheting For Dummies Revised,Wiley Publishing,0470536454
4,Staff Boards,Music Theory Staff Paper Notebook,Music Theory Shop,0692048693


In [4]:
# Tạo vectorizer object và fit chuỗi title trong dataset
vectorizer = TfidfVectorizer(max_features=100000)
vectorizer.fit(meta_data["title"])

TfidfVectorizer(max_features=100000)

In [5]:
# Dùng fitted vectorizer để transform title thành dạng số và thêm cột chứa title đã được số hóa vào data
vectorizedTitle = meta_data["title"].apply(lambda x: vectorizer.transform([x]).sum())
meta_data.insert(2, "vectorizedTitle", vectorizedTitle)
meta_data.head()

,category,title,vectorizedTitle,brand,asin
0,Guiros,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,2.823193,WADSUWAN SHOP,0000989983
1,Sound Effects,"Wooden Percussion 3 Piece Set Frog, Cricket an...",2.848613,WADSUWAN SHOP,0000098906
2,General Accessories,Hal Leonard Vivaldi Four Seasons for Piano (Or...,3.044843,Hal Leonard,0041291905
3,General Accessories,Wiley Publishers Crocheting For Dummies Revised,2.337312,Wiley Publishing,0470536454
4,Staff Boards,Music Theory Staff Paper Notebook,2.183174,Music Theory Shop,0692048693


In [6]:
# Tạo Label encoder object để số hóa category và brand
encoder = LabelEncoder()
encodedCategory = encoder.fit_transform(meta_data.category.values)
encodedBrand = encoder.fit_transform(meta_data.brand.values)

# Thêm category và brand đã được số hóa vào data
meta_data.insert(1, "encodedCategory", encodedCategory)
meta_data.insert(5, "encodedBrand", encodedBrand)
meta_data.head()

,category,encodedCategory,title,vectorizedTitle,brand,encodedBrand,asin
0,Guiros,306,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,2.823193,WADSUWAN SHOP,9905,0000989983
1,Sound Effects,565,"Wooden Percussion 3 Piece Set Frog, Cricket an...",2.848613,WADSUWAN SHOP,9905,0000098906
2,General Accessories,298,Hal Leonard Vivaldi Four Seasons for Piano (Or...,3.044843,Hal Leonard,4091,0041291905
3,General Accessories,298,Wiley Publishers Crocheting For Dummies Revised,2.337312,Wiley Publishing,10081,0470536454
4,Staff Boards,573,Music Theory Staff Paper Notebook,2.183174,Music Theory Shop,6308,0692048693


In [7]:
# Khởi tạo kNN model và fit data cần thiết vào để train model
kNN = KNeighborsClassifier(n_neighbors=20, weights="distance")
kNN.fit(meta_data[["encodedCategory", "vectorizedTitle", "encodedBrand"]], meta_data["asin"])

KNeighborsClassifier(n_neighbors=20, weights='distance')

In [8]:
# Lưu mô hình KNN với joblib
model_file = parrent_dir + "\\Best recommendation models\\KNN_CBF.pkl"  
joblib.dump(kNN, model_file)

['d:\\Source_code\\3. Hệ thống đề xuất\\Best recommendation models\\KNN_CBF.pkl']

In [9]:
def get_recommendations(itemID, k=5):
  # Truy xuất item features từ mã asin
  itemFeatures = meta_data.loc[meta_data['asin'] == itemID][["encodedCategory", "vectorizedTitle", "encodedBrand"]]

  # Tìm k sản phẩm liên quan đến asin được nhập
  distances, neighbors = kNN.kneighbors(itemFeatures)

  # Xuất sản phẩm được đề xuất và xóa sản phẩm được nhập
  recommendedItemID = pd.DataFrame(meta_data.iloc[neighbors.flatten()].asin.unique()[:k], columns=["asin"])
  recommendedItemID.drop(recommendedItemID[recommendedItemID.asin == itemID].index, inplace=True)

  return recommendedItemID.asin.tolist()

In [11]:
# Thay asin vào để chạy recommend, k là số lượng sản phẩm sẽ recommend
itemID = "B00004UE29"
k = 5

print("Sản phẩm nhập vào:")
print(meta_data[meta_data["asin"] == itemID]["title"].unique())
print()
print(f" {k} sản phẩm liên quan được đề xuất:")
itemsList = get_recommendations(itemID, k)
for i, item in enumerate(itemsList, start=1):
    print(f"{i}. " + meta_data[meta_data["asin"] == item]["title"].unique())

Sản phẩm nhập vào:
['Yamaha YRS24B Soprano Recorder']

 5 sản phẩm liên quan được đề xuất:
['1. Yamaha Recorder (YRS-302B)']
['2. Yamaha YRS402B Baroque ECODEAR']
['3. Yamaha YRS401 German Ecodear']
['4. Yamaha Alto Horn, Black (YRT304B)']
